# snapReactors

Copyright (c) Dan Kotlyar and CoRE group

# Materials Container

* This container stores materials information such as isotopic definition, abundances, uncertainties and material properties.
* The container also stores, modifies or creates new h5 files for data storage.

## Code: 

In [1]:
import numpy as np
from snapReactors.containers.materials import Material
from snapReactors.containers.property import Constant, Table, Correlation

## Defining a new material

### Define with Materials container

1. Give name of material
2. Give type of uncertainty, must be in ``Enum.UTYPE`` which are:
    - ABSOLUTE 
    - RELATIVE
    - PERCENTAGE
    - NONE
3. Give the composition type, must be in ``Enum.CTYPE`` which are:
    - RELATIVE
    - WEIGHT
    - ATOMIC
4. Give the isotopes that define a material as a np.array
5. Give the abundances for each isotope as a np.array
6. The uncertainty value, reference, description, and properties are left as optional parameters.
    1. Note that properties must be under ``ALLOWED_PROPERTIES``


In [2]:
mat1 = Material("newMat", 'NONE', 'WEIGHT', np.array([]), np.array([]), None, reference=None, description='This is an example', _properties=None)
print(mat1)

{'matName': ['newMat'], 'utype': [<UTYPE.NONE: 4>], 'ctype': [<CTYPE.WEIGHT: 2>], 'abundances': [array([], dtype=float64)], 'isotopes': [array([], dtype=float64)], 'unc': [None], 'reference': [None], 'description': ['This is an example'], '_properties': [None]}


### Reading in isotopic defintion through a text file

The ``Materials.readData(filename)`` method is used to read in all relevant materials information through a text file. There are several rules to keep in mind for the structure of the text file:
1. The ``Material Name`` must be indicated at the beginning of every material data section with the following format:
    - Material Name: Example Name
2. The ``ctype``, ``utype``, and ``Number of isotopes`` must be indicated before the beginning of ``Isotopic Definition`` although their order doesn't matter. 
3. The ``Isotopic Definition`` must have a line between itself and where the definition begins. In the example below a dashed line is used to indicate this seperation.
4. For each input a colon is used to seperate the keyword and input, for example:
    - utype: NONE
5. The location of ``Reference`` and ``Description`` for a specific material must be placed before the beginning of the next ``Material Name`` if present. 

Material Property data can be read in by adding a ``Properties`` section. 
1. The location of ``Properties`` must be before the beginning of the next ``Material Name`` and is indicated with curly brackets:
   ```
    Properties: {
        
    }
    ```
2. The formatting for the ``Properties`` information only requires there to be a colon in between the keyword and value, and a space between keywords:
    ```
    Properties: {
    type:table id:h unit:imperial ref:NAA-SR-6160 
    dep1unit:K dep1values:{300,600,900} 
    dep2unit:Pa dep2values:{16,32,48}
    value:{{1,2,3},{4,5,6},{7,8,9}}
    unc:{{.01,.01,.01},{.01,.01,.01},{.01,.01,.01}}
    }
    ```
3. Structure of ``Properties`` input is outlined in Property Container documentation.

Optional parameters such as reference or uncertainty values can be left out, however, warnings will be highlighted to the user. Two examples for the Material Property data are shown below.

####  Example text file shown below

In [3]:
text_file = open('test.txt')
file_content = text_file.read()
print(file_content)
text_file.close()

Material Name: hasteC
ctype: RELATIVE
utype: NONE
Number of isotopes: 33
Isotopic Definition:
-------------------
6000.03c      0.0007    
27059.03c     0.0125    
24050.03c     0.006952
24052.03c     0.1340624
24053.03c     0.0152016
24054.03c     0.003784
42092.03c     0.0249033
42094.03c     0.0156179
42095.03c     0.0269841
42096.03c     0.0283441
42097.03c     0.0162894
42098.03c     0.0412964
42100.03c     0.0165648
23050.03c     0.0000075
23051.03c     0.0029925
74180.03c     0.00048
74182.03c     0.106
74183.03c     0.05724
74184.03c     0.12256
74186.03c     0.11372
26054.03c     0.003360875
26056.03c     0.05275855
26057.03c     0.001218425
26058.03c     0.00016215
25055.03c     0.01     
14028.03c     0.00645561
14029.03c     0.00032795
14030.03c     0.00032795
28058.03c     0.1220600887
28060.03c     0.0470180183
28061.03c     0.0020438407
28062.03c     0.0065166585
28064.03c     0.0016596008
Properties: {
%property values for material
%type = const, table, corr
%id = prop 

#### Materials definition returned by readData

In [4]:
mats = Material.readData('test.txt')
print(mats)

[<snapReactors.containers.materials.Material object at 0x000002316F7A8160>, <snapReactors.containers.materials.Material object at 0x000002316F7CCA00>]


C:\Users\Samuel\AppData\Roaming\Python\Python38\site-packages\snapreactors-0.0.1-py3.8.egg\snapReactors\containers\property.py:844: InputFileSyntaxWarning: uncertainty not given for r corr property @ line: 18
  warnings.warn("uncertainty not given for {} {} property @"
C:\Users\Samuel\AppData\Roaming\Python\Python38\site-packages\snapreactors-0.0.1-py3.8.egg\snapReactors\containers\property.py:645: InputFileSyntaxWarning: reference not given for cp const property @ line: 9
  warnings.warn("reference not given for {} {} property @"


## Updating properties to materials

1. The properties must be from the following list: ['cp', 'cv', 'g', 'h', 'my', 'pr', 'r', 's', 'tc', 'v']

In [5]:
p1 = Constant(id='cv',  value=1, unit= "J/kg/K", unc=None, ref=None, description=None)
print(p1)

mat1.addproperty([p1])
print(mat1)


{'id': 'cv', 'dtype': <DTYPE.NUMBER: 1>, 'vtype': <VTYPE.CONSTANT: 1>, 'value': array([1]), 'valueUnit': 'J/kg/K', 'unc': None, 'dependents': None, 'dependentsUnit': None, 'description': None, 'ref': None}
{'matName': ['newMat'], 'utype': [<UTYPE.NONE: 4>], 'ctype': [<CTYPE.WEIGHT: 2>], 'abundances': [array([], dtype=float64)], 'isotopes': [array([], dtype=float64)], 'unc': [None], 'reference': [None], 'description': ['This is an example'], '_properties': [<snapReactors.containers.property.Constant object at 0x000002316F7F6E50>]}
